# Chapter 36: File Handling & Uploads

Modern applications require robust file handling capabilities—from user avatars and document uploads to media processing and bulk data imports. Implementing secure, scalable file uploads in Next.js requires careful architecture to prevent security vulnerabilities, minimize server load, and optimize storage costs. Direct-to-cloud-storage patterns allow browsers to upload files straight to S3 or R2 without consuming application server bandwidth, while server-side processing handles validation, metadata extraction, and post-processing.

By the end of this chapter, you'll master implementing presigned URL patterns for secure direct uploads, processing images with Sharp for optimization, validating file types and scanning for malware, tracking upload progress with real-time UI updates, handling large file multipart uploads, and integrating with object storage services like AWS S3 and Cloudflare R2.

## 36.1 Secure Upload Architecture

Implement presigned URL patterns to keep cloud credentials secure while enabling direct browser-to-storage uploads.

### Presigned URL Generation

```typescript
// lib/storage/s3.ts
import { S3Client, PutObjectCommand } from '@aws-sdk/client-s3';
import { getSignedUrl } from '@aws-sdk/s3-request-presigner';
import { v4 as uuidv4 } from 'uuid';

const s3Client = new S3Client({
  region: process.env.AWS_REGION,
  credentials: {
    accessKeyId: process.env.AWS_ACCESS_KEY_ID!,
    secretAccessKey: process.env.AWS_SECRET_ACCESS_KEY!,
  },
});

interface PresignedUrlOptions {
  filename: string;
  contentType: string;
  size: number;
  maxSize?: number;
  allowedTypes?: string[];
}

export async function generatePresignedUrl({
  filename,
  contentType,
  size,
  maxSize = 10 * 1024 * 1024, // 10MB default
  allowedTypes = ['image/jpeg', 'image/png', 'image/webp', 'application/pdf'],
}: PresignedUrlOptions) {
  // Server-side validation (never trust client)
  if (size > maxSize) {
    throw new Error(`File size exceeds maximum allowed (${maxSize} bytes)`);
  }
  
  if (!allowedTypes.includes(contentType)) {
    throw new Error(`File type ${contentType} not allowed`);
  }

  // Generate unique key with sanitized filename
  const extension = filename.split('.').pop()?.toLowerCase();
  const sanitized = filename.replace(/[^a-z0-9.]/gi, '-').toLowerCase();
  const key = `uploads/${uuidv4()}-${sanitized}`;

  const command = new PutObjectCommand({
    Bucket: process.env.S3_BUCKET_NAME,
    Key: key,
    ContentType: contentType,
    ContentLength: size,
    Metadata: {
      'original-name': filename,
      'uploaded-at': new Date().toISOString(),
    },
  });

  const url = await getSignedUrl(s3Client, command, { expiresIn: 300 }); // 5 minutes
  
  return {
    url,
    key,
    fields: {
      bucket: process.env.S3_BUCKET_NAME,
      key,
    },
  };
}

// Alternative for Cloudflare R2 (S3-compatible)
import { S3Client as R2Client } from '@aws-sdk/client-s3';

export const r2Client = new R2Client({
  region: 'auto',
  endpoint: `https://${process.env.R2_ACCOUNT_ID}.r2.cloudflarestorage.com`,
  credentials: {
    accessKeyId: process.env.R2_ACCESS_KEY_ID!,
    secretAccessKey: process.env.R2_SECRET_ACCESS_KEY!,
  },
});
```

### Upload Route Handler

```typescript
// app/api/upload/presigned/route.ts
import { generatePresignedUrl } from '@/lib/storage/s3';
import { NextRequest, NextResponse } from 'next/server';
import { z } from 'zod';

const uploadSchema = z.object({
  filename: z.string().min(1),
  contentType: z.string(),
  size: z.number().max(100 * 1024 * 1024), // 100MB max
});

export async function POST(req: NextRequest) {
  try {
    const body = await req.json();
    const validated = uploadSchema.parse(body);
    
    // Additional security: Check user authentication/authorization
    const user = await getCurrentUser(req);
    if (!user) {
      return NextResponse.json({ error: 'Unauthorized' }, { status: 401 });
    }

    const result = await generatePresignedUrl({
      ...validated,
      allowedTypes: ['image/jpeg', 'image/png', 'image/webp', 'image/gif', 'application/pdf'],
      maxSize: 50 * 1024 * 1024, // 50MB for images/docs
    });

    // Store pending upload in database for tracking
    await db.fileUpload.create({
      data: {
        key: result.key,
        userId: user.id,
        filename: validated.filename,
        contentType: validated.contentType,
        size: validated.size,
        status: 'pending',
        expiresAt: new Date(Date.now() + 5 * 60 * 1000), // 5 min expiry
      },
    });

    return NextResponse.json(result);
  } catch (error) {
    if (error instanceof z.ZodError) {
      return NextResponse.json({ error: 'Invalid request', details: error.errors }, { status: 400 });
    }
    console.error('Presigned URL generation failed:', error);
    return NextResponse.json({ error: 'Failed to generate upload URL' }, { status: 500 });
  }
}

// Webhook handler for S3 upload completion (optional)
// app/api/webhooks/s3-upload/route.ts
export async function POST(req: NextRequest) {
  // Verify S3/SNS signature...
  const event = await req.json();
  
  if (event.Event === 's3:ObjectCreated:Put') {
    const key = event.Records[0].s3.object.key;
    
    // Update database record to completed
    await db.fileUpload.update({
      where: { key },
      data: { 
        status: 'completed',
        completedAt: new Date(),
        url: `https://${process.env.CDN_DOMAIN}/${key}`,
      },
    });
    
    // Trigger post-processing (image optimization, virus scan, etc.)
    await queuePostProcessing(key);
  }
  
  return Response.json({ received: true });
}
```

## 36.2 Client-Side Upload Implementation

Build React hooks and components for handling file selection, validation, and progress tracking.

### Upload Hook with Progress

```typescript
// hooks/use-file-upload.ts
'use client';

import { useState, useCallback } from 'react';
import axios from 'axios';

interface UploadProgress {
  loaded: number;
  total: number;
  percentage: number;
}

interface UseFileUploadOptions {
  onSuccess?: (data: { key: string; url: string }) => void;
  onError?: (error: Error) => void;
  onProgress?: (progress: UploadProgress) => void;
}

export function useFileUpload(options: UseFileUploadOptions = {}) {
  const [isUploading, setIsUploading] = useState(false);
  const [progress, setProgress] = useState<UploadProgress | null>(null);
  const [abortController, setAbortController] = useState<AbortController | null>(null);

  const upload = useCallback(async (file: File) => {
    setIsUploading(true);
    setProgress(null);
    
    const controller = new AbortController();
    setAbortController(controller);

    try {
      // Step 1: Get presigned URL
      const presignedRes = await fetch('/api/upload/presigned', {
        method: 'POST',
        headers: { 'Content-Type': 'application/json' },
        body: JSON.stringify({
          filename: file.name,
          contentType: file.type,
          size: file.size,
        }),
      });

      if (!presignedRes.ok) {
        const error = await presignedRes.json();
        throw new Error(error.error || 'Failed to get upload URL');
      }

      const { url, key } = await presignedRes.json();

      // Step 2: Upload to S3/R2 directly
      await axios.put(url, file, {
        headers: {
          'Content-Type': file.type,
        },
        signal: controller.signal,
        onUploadProgress: (progressEvent) => {
          const loaded = progressEvent.loaded;
          const total = progressEvent.total || file.size;
          const percentage = Math.round((loaded * 100) / total);
          
          const progressData = { loaded, total, percentage };
          setProgress(progressData);
          options.onProgress?.(progressData);
        },
      });

      // Step 3: Confirm upload (optional, depending on architecture)
      const confirmRes = await fetch('/api/upload/confirm', {
        method: 'POST',
        headers: { 'Content-Type': 'application/json' },
        body: JSON.stringify({ key }),
      });

      const result = await confirmRes.json();
      
      options.onSuccess?.({ key, url: result.url });
      return { key, url: result.url };
    } catch (error) {
      if (axios.isCancel(error)) {
        console.log('Upload cancelled');
      } else {
        options.onError?.(error as Error);
      }
      throw error;
    } finally {
      setIsUploading(false);
      setAbortController(null);
    }
  }, [options]);

  const cancel = useCallback(() => {
    abortController?.abort();
    setIsUploading(false);
    setProgress(null);
  }, [abortController]);

  return {
    upload,
    cancel,
    isUploading,
    progress,
  };
}
```

### Dropzone Component

```typescript
// components/file-upload/dropzone.tsx
'use client';

import { useCallback, useState } from 'react';
import { useDropzone } from 'react-dropzone';
import { useFileUpload } from '@/hooks/use-file-upload';
import { Progress } from '@/components/ui/progress';
import { FileIcon, XIcon, UploadCloudIcon } from 'lucide-react';

interface FileDropzoneProps {
  accept?: Record<string, string[]>;
  maxSize?: number;
  maxFiles?: number;
  onUploadComplete?: (files: Array<{ key: string; url: string }>) => void;
}

export function FileDropzone({
  accept = { 'image/*': ['.png', '.jpg', '.jpeg', '.webp', '.gif'] },
  maxSize = 10 * 1024 * 1024,
  maxFiles = 5,
  onUploadComplete,
}: FileDropzoneProps) {
  const [files, setFiles] = useState<Array<File & { id: string; status: 'pending' | 'uploading' | 'done' | 'error'; progress?: number }>>([]);
  
  const onDrop = useCallback((acceptedFiles: File[]) => {
    const newFiles = acceptedFiles.map(file => ({
      ...file,
      id: Math.random().toString(36).substring(7),
      status: 'pending' as const,
    }));
    
    setFiles(prev => [...prev, ...newFiles].slice(0, maxFiles));
    
    // Auto-upload
    newFiles.forEach(uploadFile);
  }, []);

  const { getRootProps, getInputProps, isDragActive, fileRejections } = useDropzone({
    onDrop,
    accept,
    maxSize,
    maxFiles,
  });

  const uploadFile = async (file: any) => {
    setFiles(prev => prev.map(f => 
      f.id === file.id ? { ...f, status: 'uploading' } : f
    ));

    try {
      // Use the upload hook logic here
      const formData = new FormData();
      formData.append('file', file);
      
      // Simplified for example - use the presigned URL pattern from above
      const response = await fetch('/api/upload/direct', {
        method: 'POST',
        body: formData,
      });
      
      if (!response.ok) throw new Error('Upload failed');
      
      const result = await response.json();
      
      setFiles(prev => prev.map(f => 
        f.id === file.id ? { ...f, status: 'done', url: result.url } : f
      ));
      
      onUploadComplete?.([result]);
    } catch (error) {
      setFiles(prev => prev.map(f => 
        f.id === file.id ? { ...f, status: 'error' } : f
      ));
    }
  };

  const removeFile = (id: string) => {
    setFiles(prev => prev.filter(f => f.id !== id));
  };

  return (
    <div className="space-y-4">
      <div
        {...getRootProps()}
        className={`border-2 border-dashed rounded-lg p-8 text-center cursor-pointer transition-colors ${
          isDragActive ? 'border-blue-500 bg-blue-50' : 'border-gray-300 hover:border-gray-400'
        }`}
      >
        <input {...getInputProps()} />
        <UploadCloudIcon className="w-12 h-12 mx-auto text-gray-400 mb-4" />
        <p className="text-gray-600">
          {isDragActive ? 'Drop the files here...' : 'Drag & drop files here, or click to select'}
        </p>
        <p className="text-sm text-gray-400 mt-2">
          Max size: {(maxSize / 1024 / 1024).toFixed(1)}MB • Max files: {maxFiles}
        </p>
      </div>

      {fileRejections.length > 0 && (
        <div className="p-4 bg-red-50 text-red-600 rounded-lg text-sm">
          {fileRejections.map(({ file, errors }) => (
            <div key={file.name}>
              {file.name}: {errors.map(e => e.message).join(', ')}
            </div>
          ))}
        </div>
      )}

      {files.length > 0 && (
        <div className="space-y-2">
          {files.map((file) => (
            <div key={file.id} className="flex items-center gap-4 p-3 bg-gray-50 rounded-lg">
              <FileIcon className="w-8 h-8 text-gray-400" />
              
              <div className="flex-1 min-w-0">
                <p className="text-sm font-medium truncate">{file.name}</p>
                <p className="text-xs text-gray-500">
                  {(file.size / 1024).toFixed(1)} KB
                </p>
                
                {file.status === 'uploading' && (
                  <Progress value={file.progress} className="h-1 mt-2" />
                )}
              </div>

              <div className="flex items-center gap-2">
                {file.status === 'done' && (
                  <span className="text-green-600 text-sm">✓ Done</span>
                )}
                {file.status === 'error' && (
                  <span className="text-red-600 text-sm">Failed</span>
                )}
                
                <button
                  onClick={() => removeFile(file.id)}
                  className="p-1 hover:bg-gray-200 rounded"
                  aria-label="Remove file"
                >
                  <XIcon className="w-4 h-4" />
                </button>
              </div>
            </div>
          ))}
        </div>
      )}
    </div>
  );
}
```

## 36.3 Image Processing and Optimization

Process uploaded images to generate thumbnails, responsive variants, and optimize formats.

### Sharp Processing Pipeline

```typescript
// lib/image-processing.ts
import sharp from 'sharp';
import { S3Client, GetObjectCommand, PutObjectCommand } from '@aws-sdk/client-s3';
import { Readable } from 'stream';

const s3 = new S3Client({ region: process.env.AWS_REGION });

interface ProcessOptions {
  widths?: number[];
  formats?: ('jpeg' | 'webp' | 'avif')[];
  quality?: number;
}

export async function processImageUpload(key: string, options: ProcessOptions = {}) {
  const { widths = [640, 1080, 1920], formats = ['webp', 'jpeg'], quality = 85 } = options;

  // Fetch original from S3
  const getCommand = new GetObjectCommand({
    Bucket: process.env.S3_BUCKET_NAME,
    Key: key,
  });
  
  const response = await s3.send(getCommand);
  const stream = response.Body as Readable;
  
  // Convert stream to buffer
  const chunks: Buffer[] = [];
  for await (const chunk of stream) {
    chunks.push(chunk);
  }
  const buffer = Buffer.concat(chunks);

  // Get metadata
  const metadata = await sharp(buffer).metadata();
  const results: Array<{ width: number; format: string; key: string; url: string }> = [];

  // Generate variants
  for (const width of widths) {
    // Skip if original is smaller than target width
    if (metadata.width && metadata.width < width) continue;

    for (const format of formats) {
      const resized = await sharp(buffer)
        .resize(width, null, { 
          withoutEnlargement: true,
          fit: 'inside',
        })
        .toFormat(format, { quality })
        .toBuffer();

      const variantKey = key.replace(/(\.[^.]+)$/, `-${width}.${format}`);
      
      await s3.send(new PutObjectCommand({
        Bucket: process.env.S3_BUCKET_NAME,
        Key: variantKey,
        Body: resized,
        ContentType: `image/${format}`,
        Metadata: {
          'original-key': key,
          'width': width.toString(),
          'format': format,
        },
      }));

      results.push({
        width,
        format,
        key: variantKey,
        url: `https://${process.env.CDN_DOMAIN}/${variantKey}`,
      });
    }
  }

  // Generate blur placeholder
  const blurPlaceholder = await sharp(buffer)
    .resize(20, null, { fit: 'inside' })
    .blur()
    .toBuffer();
    
  const blurBase64 = `data:image/jpeg;base64,${blurPlaceholder.toString('base64')}`;

  return {
    original: key,
    variants: results,
    blurDataUrl: blurBase64,
    aspectRatio: metadata.width && metadata.height ? metadata.width / metadata.height : null,
  };
}

// Server Action for image processing
// app/actions/process-image.ts
'use server';

import { processImageUpload } from '@/lib/image-processing';
import { revalidateTag } from 'next/cache';

export async function processUploadedImage(key: string) {
  try {
    const result = await processImageUpload(key, {
      widths: [400, 800, 1200],
      formats: ['webp', 'jpeg'],
    });
    
    // Update database with processed image info
    await db.image.update({
      where: { key },
      data: {
        processed: true,
        variants: result.variants,
        blurDataUrl: result.blurDataUrl,
        aspectRatio: result.aspectRatio,
      },
    });
    
    revalidateTag(`image-${key}`);
    
    return { success: true, variants: result.variants };
  } catch (error) {
    console.error('Image processing failed:', error);
    throw new Error('Failed to process image');
  }
}
```

## 36.4 File Validation and Security

Implement comprehensive security measures including MIME type verification and malware scanning.

### Security Utilities

```typescript
// lib/file-security.ts
import { fileTypeFromBuffer } from 'file-type';
import magicBytes from 'magic-bytes.js';
import { spawn } from 'child_process';
import { promisify } from 'util';
import { writeFile, unlink } from 'fs/promises';
import path from 'path';
import { tmpdir } from 'os';

// Magic numbers validation (more secure than extension checking)
export async function validateFileType(buffer: Buffer, allowedTypes: string[]): Promise<boolean> {
  // Use file-type library for modern formats
  const type = await fileTypeFromBuffer(buffer);
  
  if (!type) {
    // Fallback to magic-bytes for additional formats
    const detected = magicBytes(buffer);
    if (detected.length === 0) return false;
    return allowedTypes.includes(detected[0].typename);
  }
  
  return allowedTypes.includes(type.mime);
}

// ClamAV Virus Scanning
export async function scanFile(buffer: Buffer): Promise<{ clean: boolean; viruses?: string[] }> {
  if (process.env.DISABLE_VIRUS_SCAN === 'true') {
    return { clean: true };
  }

  const tempPath = path.join(tmpdir(), `scan-${Date.now()}`);
  
  try {
    await writeFile(tempPath, buffer);
    
    return new Promise((resolve, reject) => {
      const clamscan = spawn('clamdscan', ['--no-summary', '--stdout', tempPath]);
      let output = '';
      
      clamscan.stdout.on('data', (data) => {
        output += data.toString();
      });
      
      clamscan.on('close', async (code) => {
        await unlink(tempPath);
        
        if (code === 0) {
          resolve({ clean: true });
        } else if (code === 1) {
          // Virus found
          const viruses = output.match(/FOUND\s+(.+)/)?.[1];
          resolve({ clean: false, viruses: viruses ? [viruses] : ['Unknown'] });
        } else {
          reject(new Error('Virus scan failed'));
        }
      });
      
      clamscan.on('error', (err) => {
        unlink(tempPath).catch(console.error);
        reject(err);
      });
    });
  } catch (error) {
    await unlink(tempPath).catch(() => {});
    throw error;
  }
}

// Content Security Policy for uploads
export function getUploadSecurityHeaders(contentType: string): Record<string, string> {
  // Force download for executable files
  const dangerousTypes = ['application/x-msdownload', 'application/x-executable'];
  
  if (dangerousTypes.some(t => contentType.includes(t))) {
    return {
      'Content-Disposition': 'attachment',
      'X-Content-Type-Options': 'nosniff',
    };
  }
  
  return {
    'X-Content-Type-Options': 'nosniff',
  };
}

// Sanitize filename
export function sanitizeFilename(filename: string): string {
  // Remove path traversal attempts
  const basename = path.basename(filename);
  // Remove control characters
  const sanitized = basename.replace(/[\x00-\x1f\x80-\x9f]/g, '');
  // Limit length
  return sanitized.slice(0, 255);
}
```

### Secure File Serving

```typescript
// app/api/files/[...key]/route.ts
import { NextRequest, NextResponse } from 'next/server';
import { S3Client, GetObjectCommand, HeadObjectCommand } from '@aws-sdk/client-s3';
import { getSignedUrl } from '@aws-sdk/s3-request-presigner';

const s3 = new S3Client({ region: process.env.AWS_REGION });

export async function GET(
  req: NextRequest,
  { params }: { params: { key: string[] } }
) {
  const key = params.key.join('/');
  
  try {
    // Check if file exists and get metadata
    const headCommand = new HeadObjectCommand({
      Bucket: process.env.S3_BUCKET_NAME,
      Key: key,
    });
    
    const metadata = await s3.send(headCommand);
    
    // Security checks
    const contentType = metadata.ContentType || 'application/octet-stream';
    
    // Block dangerous file types from being served inline
    const blockedTypes = [
      'application/x-msdownload',
      'application/x-executable',
      'text/html',
      'application/javascript',
    ];
    
    if (blockedTypes.includes(contentType)) {
      return new NextResponse('File type not allowed', { status: 403 });
    }

    // Check user authorization (example: private files)
    const user = await getCurrentUser(req);
    const fileRecord = await db.fileUpload.findUnique({ where: { key } });
    
    if (fileRecord?.userId && fileRecord.userId !== user?.id) {
      return new NextResponse('Unauthorized', { status: 403 });
    }

    // Generate presigned URL for private files, or redirect to CDN for public
    if (fileRecord?.isPrivate) {
      const getCommand = new GetObjectCommand({
        Bucket: process.env.S3_BUCKET_NAME,
        Key: key,
        ResponseContentDisposition: `inline; filename="${fileRecord.filename}"`,
      });
      
      const url = await getSignedUrl(s3, getCommand, { expiresIn: 300 });
      return NextResponse.redirect(url);
    }
    
    // Public file - redirect to CDN
    return NextResponse.redirect(`https://${process.env.CDN_DOMAIN}/${key}`);
    
  } catch (error) {
    if ((error as any).name === 'NoSuchKey') {
      return new NextResponse('File not found', { status: 404 });
    }
    return new NextResponse('Internal error', { status: 500 });
  }
}
```

## 36.5 Large File and Resumable Uploads

Handle large files with multipart uploads and support resumable uploads for unreliable connections.

### Multipart Upload Handler

```typescript
// lib/storage/multipart.ts
import { S3Client, CreateMultipartUploadCommand, UploadPartCommand, CompleteMultipartUploadCommand, AbortMultipartUploadCommand } from '@aws-sdk/client-s3';

const s3 = new S3Client({ region: process.env.AWS_REGION });
const PART_SIZE = 5 * 1024 * 1024; // 5MB parts (minimum for S3)

export async function initiateMultipartUpload(filename: string, contentType: string) {
  const key = `uploads/large/${Date.now()}-${sanitizeFilename(filename)}`;
  
  const command = new CreateMultipartUploadCommand({
    Bucket: process.env.S3_BUCKET_NAME,
    Key: key,
    ContentType: contentType,
  });
  
  const response = await s3.send(command);
  
  // Store multipart upload ID in database
  const uploadRecord = await db.multipartUpload.create({
    data: {
      uploadId: response.UploadId!,
      key,
      filename,
      contentType,
      status: 'initiated',
      parts: [],
    },
  });
  
  return {
    uploadId: response.UploadId,
    key,
    partSize: PART_SIZE,
  };
}

export async function getPresignedPartUrl(uploadId: string, key: string, partNumber: number) {
  const command = new UploadPartCommand({
    Bucket: process.env.S3_BUCKET_NAME,
    Key: key,
    UploadId: uploadId,
    PartNumber: partNumber,
  });
  
  // Note: AWS SDK v3 doesn't support presigned URLs for multipart directly
  // You'd typically use @aws-sdk/s3-request-presigner or handle parts server-side
  
  return { url: 'presigned-url-placeholder', partNumber };
}

export async function completeMultipartUpload(uploadId: string, key: string, parts: Array<{ ETag: string; PartNumber: number }>) {
  const command = new CompleteMultipartUploadCommand({
    Bucket: process.env.S3_BUCKET_NAME,
    Key: key,
    UploadId: uploadId,
    MultipartUpload: { Parts: parts.sort((a, b) => a.PartNumber - b.PartNumber) },
  });
  
  await s3.send(command);
  
  await db.multipartUpload.update({
    where: { uploadId },
    data: { status: 'completed', completedAt: new Date() },
  });
  
  return { key, url: `https://${process.env.CDN_DOMAIN}/${key}` };
}
```

### Client-Side Resumable Upload

```typescript
// hooks/use-resumable-upload.ts
'use client';

import { useState, useCallback, useRef } from 'react';

interface Part {
  number: number;
  etag?: string;
  status: 'pending' | 'uploading' | 'complete' | 'error';
}

export function useResumableUpload() {
  const [progress, setProgress] = useState(0);
  const [isUploading, setIsUploading] = useState(false);
  const abortController = useRef<AbortController | null>(null);
  const parts = useRef<Part[]>([]);

  const uploadLargeFile = useCallback(async (file: File) => {
    setIsUploading(true);
    setProgress(0);
    
    try {
      // 1. Initiate multipart upload
      const initRes = await fetch('/api/upload/multipart/init', {
        method: 'POST',
        headers: { 'Content-Type': 'application/json' },
        body: JSON.stringify({
          filename: file.name,
          contentType: file.type,
          size: file.size,
        }),
      });
      
      const { uploadId, key, partSize } = await initRes.json();
      
      // 2. Calculate parts
      const totalParts = Math.ceil(file.size / partSize);
      parts.current = Array.from({ length: totalParts }, (_, i) => ({
        number: i + 1,
        status: 'pending',
      }));
      
      // 3. Upload parts (in parallel with concurrency limit)
      const CONCURRENCY = 3;
      const queue: Promise<void>[] = [];
      
      for (let i = 0; i < totalParts; i++) {
        const partNumber = i + 1;
        const start = i * partSize;
        const end = Math.min(start + partSize, file.size);
        const chunk = file.slice(start, end);
        
        const uploadPromise = uploadPart(uploadId, key, partNumber, chunk, file.type);
        queue.push(uploadPromise);
        
        if (queue.length >= CONCURRENCY) {
          await Promise.all(queue);
          queue.length = 0;
        }
      }
      
      await Promise.all(queue);
      
      // 4. Complete upload
      const completeRes = await fetch('/api/upload/multipart/complete', {
        method: 'POST',
        headers: { 'Content-Type': 'application/json' },
        body: JSON.stringify({
          uploadId,
          key,
          parts: parts.current.map(p => ({ ETag: p.etag, PartNumber: p.number })),
        }),
      });
      
      return await completeRes.json();
    } catch (error) {
      console.error('Resumable upload failed:', error);
      throw error;
    } finally {
      setIsUploading(false);
    }
  }, []);

  const uploadPart = async (uploadId: string, key: string, partNumber: number, chunk: Blob, contentType: string) => {
    // Get presigned URL for this part
    const urlRes = await fetch(`/api/upload/multipart/url?uploadId=${uploadId}&key=${key}&part=${partNumber}`);
    const { url } = await urlRes.json();
    
    // Upload part
    const response = await fetch(url, {
      method: 'PUT',
      body: chunk,
      headers: { 'Content-Type': contentType },
    });
    
    if (!response.ok) throw new Error(`Part ${partNumber} failed`);
    
    const etag = response.headers.get('ETag');
    parts.current[partNumber - 1].etag = etag?.replace(/"/g, '');
    parts.current[partNumber - 1].status = 'complete';
    
    // Update progress
    const completed = parts.current.filter(p => p.status === 'complete').length;
    setProgress(Math.round((completed / parts.current.length) * 100));
  };

  return {
    uploadLargeFile,
    progress,
    isUploading,
  };
}
```

## Key Takeaways from Chapter 36

1. **Presigned URL Pattern**: Never expose cloud storage credentials to the client. Generate time-limited presigned URLs server-side that allow direct browser-to-S3 uploads. This offloads bandwidth from your application servers while maintaining security through server-side validation of file types, sizes, and user permissions before generating URLs.

2. **Client-Side Uploads**: Use React hooks with `XMLHttpRequest` or Axios to track upload progress. Implement abort controllers for cancellation capabilities. Validate file types client-side for UX, but always re-validate server-side before generating presigned URLs.

3. **Image Processing**: Use Sharp for server-side image optimization—generating responsive variants (400w, 800w, 1200w), converting to modern formats (WebP, AVIF), and creating blur placeholders for Next.js Image components. Process images asynchronously via queues (Redis/Bull) to avoid blocking API responses.

4. **Security Measures**: Validate file types using magic numbers (file-type library) rather than extensions alone. Scan uploaded files with ClamAV or similar antivirus tools before marking uploads as complete. Sanitize filenames to prevent path traversal attacks. Serve files with `X-Content-Type-Options: nosniff` and appropriate Content-Disposition headers.

5. **Large File Handling**: For files over 100MB, implement multipart (chunked) uploads with 5MB+ parts. Track part ETags client-side and complete the multipart upload once all parts succeed. Store upload progress in Redis to enable resumable uploads across page refreshes.

6. **Storage Architecture**: Use S3-compatible storage (AWS S3, Cloudflare R2, MinIO) with CDN fronting (CloudFront or Cloudflare) for global distribution. Configure CORS policies to allow direct browser uploads only from your domain. Enable object versioning to prevent accidental overwrites.

7. **Post-Processing**: Trigger image optimization, virus scanning, and metadata extraction via webhooks or queue workers after upload completion. Use database records to track upload status (pending → processing → complete) and maintain references between user records and storage keys.

## Coming Up Next

**Chapter 37: Development Tooling**

With file handling capabilities secured, it's time to optimize your development workflow. In Chapter 37, we'll explore advanced VS Code configurations for Next.js, ESLint and Prettier setups for consistent code quality, Git hooks for pre-commit validation, debugging techniques for Server Components, and development best practices. You'll learn how to configure a professional-grade development environment that catches errors early and maintains code quality across your team.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='35. e-commerce_integration.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../7. Tooling_and_workflow/37. development_tooling.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
